<a href="https://colab.research.google.com/github/yashbagadiya1997/IMAGE-CLASSIFIER-AIPY-/blob/main/testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/FLOWERS102_IMAGECLASSIFIER_UDACITY

cat_to_name.json  test	train  valid


In [ ]:
# Imports here
import torch
import numpy as np
from torch import nn
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from torchvision import models
from collections import OrderedDict
import torch.optim as optim




In [ ]:
# TODO: Define your transforms for the training, validation, and testing sets
data_transforms = transforms.Compose([transforms.RandomRotation(30),
                                      transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
val_transforms = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                     #     transforms.Resize((224, 224))
                                    ])
test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.RandomResizedCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                     ])
# TODO: Load the datasets with ImageFolder
# image_datasets = datasets.ImageFolder('flowers', transform=data_transforms)
train_data = datasets.ImageFolder('/content/drive/MyDrive/FLOWERS102_IMAGECLASSIFIER_UDACITY/train', transform=data_transforms)
valid_data = datasets.ImageFolder('/content/drive/MyDrive/FLOWERS102_IMAGECLASSIFIER_UDACITY/valid', transform=data_transforms)
test_data = datasets.ImageFolder('/content/drive/MyDrive/FLOWERS102_IMAGECLASSIFIER_UDACITY/test', transform=data_transforms)



# TODO: Using the image datasets and the trainforms, define the dataloaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=32)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32)
# dataloaders = torch.utils.data.DataLoader(image_datasets, batch_size=32, shuffle=True)

image_datasets = [train_data, valid_data, test_data]
data_loader = [train_loader, valid_loader, test_loader]


# for images, labels in train_loader:
#     print(images.shape)
#     break
# print(image_datasets)



In [ ]:
train_data

Dataset ImageFolder
    Number of datapoints: 6552
    Root location: /content/drive/MyDrive/FLOWERS102_IMAGECLASSIFIER_UDACITY/train
    StandardTransform
Transform: Compose(
               RandomRotation(degrees=[-30.0, 30.0], interpolation=nearest, expand=False, fill=0)
               RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=warn)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [ ]:
import json

with open('/content/drive/MyDrive/FLOWERS102_IMAGECLASSIFIER_UDACITY/cat_to_name.json', 'r') as f:

  cat_to_name = json.load(f)

In [ ]:
model = models.vgg16(pretrained=True)
print(model)

for param in model.parameters():
    param.requires_grad = False

model.classifier

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:05<00:00, 96.1MB/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [ ]:
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, 102)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.classifier.parameters(), lr=0.001)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
device

device(type='cuda')

In [ ]:
len(train_loader)

205

In [ ]:
for epoch in range(16):
    model.train()
    print(f"epoch :::::: {epoch}")
    train_loss = 0.0
    total_train_correct = 0  # Track total correct predictions on training data
    total_train_samples = 0  # Track total samples in the training data

    for ind, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device, dtype=torch.float32), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()
        train_loss += loss.item() * inputs.size(0)

        # Calculate training accuracy
        _, predicted = torch.max(outputs, 1)
        total_train_correct += (predicted == labels).sum().item()
        total_train_samples += labels.size(0)

    epoch_loss =  train_loss / len(train_loader.dataset)
    train_accuracy = total_train_correct / total_train_samples  # Calculate training accuracy
    print(f"Training loss: {epoch_loss:.4f}, Accuracy: {train_accuracy:.4f}")

    model.eval()
    valid_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            valid_loss += loss.item() * inputs.size(0)

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    valid_loss = valid_loss / len(valid_loader.dataset)
    accuracy = correct / total
    print(f"Validation loss: {valid_loss:.4f}, Accuracy: {accuracy:.4f}")

epoch :::::: 0
Training loss: 4.4659, Accuracy: 0.0543
Validation loss: 4.1825, Accuracy: 0.1308
epoch :::::: 1
Training loss: 4.0736, Accuracy: 0.1439
Validation loss: 3.8520, Accuracy: 0.2323
epoch :::::: 2
Training loss: 3.7565, Accuracy: 0.2199
Validation loss: 3.5965, Accuracy: 0.2689
epoch :::::: 3
Training loss: 3.5018, Accuracy: 0.2711
Validation loss: 3.3274, Accuracy: 0.3545
epoch :::::: 4
Training loss: 3.2716, Accuracy: 0.3260
Validation loss: 3.1369, Accuracy: 0.3924
epoch :::::: 5
Training loss: 3.0898, Accuracy: 0.3738
Validation loss: 2.9703, Accuracy: 0.4279
epoch :::::: 6
Training loss: 2.9310, Accuracy: 0.3996
Validation loss: 2.8151, Accuracy: 0.4645
epoch :::::: 7
Training loss: 2.7960, Accuracy: 0.4274
Validation loss: 2.7066, Accuracy: 0.4866
epoch :::::: 8
Training loss: 2.6571, Accuracy: 0.4557
Validation loss: 2.5334, Accuracy: 0.5269
epoch :::::: 9
Training loss: 2.5634, Accuracy: 0.4722
Validation loss: 2.4571, Accuracy: 0.5477
epoch :::::: 10
Training loss: